# AWS Lambda Fashion Classification

- [Notebook](notebooks/python/machine-learning/fashion-classification.md)


Install packages


In [27]:
!uv pip install -q \
    tensorflow==2.16.1 \
    tf2onnx==1.16.1 \
    onnxruntime==1.17.0 \
    keras-image-helper==0.0.1

Import packages


In [ ]:
import onnxruntime as ort
import tensorflow as tf
import tf2onnx
from keras_image_helper import create_preprocessor

Copy model binary


In [ ]:
!cp ../../machine-learning/xception_v4_37_0.891.h5 .

Load model


In [ ]:
model = tf.keras.models.load_model("./xception_v4_37_0.891.h5", compile=False)

2026-01-04 18:22:40.540469: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12582912 exceeds 10% of free system memory.
2026-01-04 18:22:40.550569: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12582912 exceeds 10% of free system memory.
2026-01-04 18:22:40.556005: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12582912 exceeds 10% of free system memory.
2026-01-04 18:22:41.215791: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12582912 exceeds 10% of free system memory.


Convert into ONNX (Open Neural Network Exchange)


In [ ]:
spec = (tf.TensorSpec((None, 299, 299, 3), tf.float32, name="input"),)

model_proto, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=spec,
    opset=16,
    output_path="clothing-classification.onnx",
)

2026-01-04 18:24:01.123970: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2026-01-04 18:24:01.124307: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2026-01-04 18:24:05.811181: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2026-01-04 18:24:05.811396: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


Create an onnx runtime session


In [ ]:
onnx_model_path = "clothing-classification.onnx"
session = ort.InferenceSession(
    onnx_model_path, providers=["CPUExecutionProvider"]
)

Get inputs and outputs


In [ ]:
inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name

print(input_name, output_name)

input dense_5


Set image for prediction path


In [ ]:
url = "https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg"

Preprocess input


In [ ]:
preprocessor = create_preprocessor("xception", target_size=(299, 299))
X = preprocessor.from_url(url)

Predictions


In [ ]:
preds = session.run([output_name], {input_name: X})
predictions_list = preds[0][0].tolist()
predictions_list

[-6.510375022888184,
 -7.7202630043029785,
 -7.819637298583984,
 -4.351745128631592,
 13.454629898071289,
 -5.087947368621826,
 -5.142486095428467,
 3.6792993545532227,
 -5.2275238037109375,
 -8.592391014099121]

Get classes


In [ ]:
classes = [
    "dress",
    "hat",
    "longsleeve",
    "outwear",
    "pants",
    "shirt",
    "shoes",
    "shorts",
    "skirt",
    "t-shirt",
]

dict(zip(classes, predictions_list))

{'dress': -6.510375022888184,
 'hat': -7.7202630043029785,
 'longsleeve': -7.819637298583984,
 'outwear': -4.351745128631592,
 'pants': 13.454629898071289,
 'shirt': -5.087947368621826,
 'shoes': -5.142486095428467,
 'shorts': 3.6792993545532227,
 'skirt': -5.2275238037109375,
 't-shirt': -8.592391014099121}